# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [4]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'nba_hyperdrive'

experiment=Experiment(ws, experiment_name)

In [3]:
# Read Data
dataset = Dataset.get_by_name(ws, name='nba-games-data')
df = dataset.to_pandas_dataframe()

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.15)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [9]:
from azureml.core.compute import ComputeTarget, AmlCompute

compute_name = 'nba-compute'
compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS2_V2', max_nodes=5)
train_cluster = ComputeTarget.create(ws, compute_name, compute_config)
train_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        '--C': choice(0.01, 0.1, 1),
        '--max_iter': choice(10, 20, 50, 100, 250)
    }
)

#TODO: Create your estimator and hyperdrive config
sklearn_env = Environment.from_conda_specification(
    name='sklearn_env',
    file_path='conda_dependencies.yml'
)

estimator = ScriptRunConfig(
    source_directory='.',
    command=['python', 'train.py'],
    environment=sklearn_env,
    compute_target=train_cluster
)

hyperdrive_run_config = HyperDriveConfig(
    run_config=estimator,
    policy=early_termination_policy,
    hyperparameter_sampling=param_sampling,
    primary_metric_name='AUC_weighted',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=5,
    max_concurrent_runs=5
)

In [ ]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run, fitted_model = hyperdrive_run.get_output()
best_run.get_metrics()

In [ ]:
#TODO: Save the best model
import joblib
joblib.dump(fitted_auto_model, 'nba-games-model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

